<figure>
  <img src="https://github.com/v-iashin/video_features/raw/master/docs/_assets/r21d.png" width="300" />
</figure>

The `video_features` library allows you to extract features from
raw videos in parallel with multiple GPUs.
It supports several extractors that capture visual appearance,
optical flow, and audio features. See more details in the
[GitHub repository](https://github.com/v-iashin/video_features).

See more feature extraction examples in colaboratory notebooks:
* [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1Zd7r8uKGLGSxlil4PPnXk_4I3KOsjPpO?usp=sharing) – CLIP
* [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1HUlYcOJf_dArOcAaR9jaQHuM5CAZiNZc?usp=sharing) – S3D
* [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1LKoytZmNxtC-EuCp7pHDM6sFvK1XdwlW?usp=sharing) – I3D
* [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1csJgkVQ3E2qOyVlcOM-ACHGgPBBKwE2Y?usp=sharing) – R(2+1)D
* [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/18I95Rn1B3a2ISfD9b-o4o93m3XuHbcIY?usp=sharing) – RAFT
* [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/17VLdf4abQT2eoMjc6ziJ9UaRaOklTlP0?usp=sharing) – ResNet
* [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1r_8OnmwXKwmH0n4RxBfuICVBgpbJt_Fs?usp=sharing) – VGGish
* [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/16QEwNMqiwqlmBhJCJmNeeEP8gitom0I-?usp=sharing) – [timm](https://huggingface.co/timm) models

In [3]:
from models.r21d.extract_r21d import ExtractR21D
from utils.utils import build_cfg_path
from omegaconf import OmegaConf
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 4070 Ti'

In [30]:
import os

directory = r"C:\Users\gonza\Downloads\train_subset\train_subset"


videos = []

for root, _, files in os.walk(directory):
    for file in files:
        if file.endswith(".mp4"):
            videos.append(os.path.join(root, file))



In [32]:
import pandas as pd
from tqdm import tqdm
# Select the feature type
feature_type = 'r21d'

# Load and patch the config
args = OmegaConf.load(build_cfg_path(feature_type))
args.video_paths = videos
args.model_name = 'r2plus1d_18_16_kinetics'  # default - Acc@1: 57.50, Acc@5: 78.81
# args.model_name = 'r2plus1d_34_32_ig65m_ft_kinetics'  # Acc@1: 79.1, Acc@5: 93.9
# args.model_name = 'r2plus1d_34_8_ig65m_ft_kinetics'
# args.show_pred = True
# args.stack_size = 24
# args.step_size = 24

# Load the model
extractor = ExtractR21D(args)

successful_features = []
failed_videos = []

for video_path in tqdm(args.video_paths, desc="Extracting features"):
    try:
        feature_dict = extractor.extract(video_path)
        for k, v in feature_dict.items():
            flattened_features = v.flatten()
            successful_features.append(flattened_features)
    except Exception as e:
        print(f"Error extracting features from {video_path}: {e}")
        failed_videos.append(video_path)

results_df = pd.DataFrame(successful_features)

results_df.to_csv('video_features.csv', index=False, header=False) 

print("Feature extraction complete.")
print(f"Successfully extracted features from {len(successful_features)} videos.")
print(f"Failed to extract features from {len(failed_videos)} videos.")

Extracting features:   3%|▎         | 363/10717 [06:25<3:26:55,  1.20s/it]

Error extracting features from C:\Users\gonza\Downloads\train_subset\train_subset\1azVHxhCCU0_000011_000021.mp4: Non-empty 4D data tensor expected but got a tensor with sizes [3, 0, 1, 1]


Extracting features:   4%|▍         | 404/10717 [07:08<4:26:52,  1.55s/it]

Error extracting features from C:\Users\gonza\Downloads\train_subset\train_subset\1IQCtz7ZUzo_000041_000051.mp4: Non-empty 4D data tensor expected but got a tensor with sizes [3, 0, 1, 1]


Extracting features:  10%|▉         | 1051/10717 [18:06<2:43:10,  1.01s/it]

...

Error extracting features from C:\Users\gonza\Downloads\train_subset\train_subset\u5zCPI91HLo_000041_000051.mp4: Non-empty 4D data tensor expected but got a tensor with sizes [3, 0, 1, 1]


Extracting features:  86%|████████▌ | 9205/10717 [2:35:15<23:08,  1.09it/s]

Error extracting features from C:\Users\gonza\Downloads\train_subset\train_subset\Vpvs1HfG2PA_000093_000103.mp4: Non-empty 4D data tensor expected but got a tensor with sizes [3, 0, 1, 1]


Extracting features: 100%|██████████| 10717/10717 [3:01:12<00:00,  1.01s/it]


Feature extraction complete.
Successfully extracted features from 10695 videos.
Failed to extract features from 22 videos. Paths saved in failed_videos array.
